# Part 4: Classification

In this notebook, we will exploit quantitative measurements in order to retreive different phenotypic groups present in a large image dataset. The end goal will be to group samples (whether full images or individual objects) into different classes, a process referred to as *classification*. We will here review commonly-used (non-deep-learning) strategies for classification.

In [ ]:
import os
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 200

In this part, we will use pandas to handle numerical data (https://pandas.pydata.org/) and seaborn to generate cute plots (). Feel free to consult the extensive documentation available on their websites if you want to know more about these libraries!

In [ ]:
import pandas as pd
import seaborn as sb

## 1. Data loading

Following on what we did in notebook 3 - Quantification, we will again work with feature matrices extracted from the BBBC010 dataset featuring dead and live *C. elegans* worms. 

**1.1** Run the lines below to load and display the feature matrix for the entire BBBC010 dataset. Note that features are here reported *per-image*: they correspond to the average value of any given feature across all instances present in the image.

In [ ]:
bbbc010_img_feats = pd.read_csv('data/Part 4/BBBC010/bbbc010_image_features.csv')
bbbc010_img_feats.set_index('image_id', inplace = True)

display(bbbc010_img_feats)

**1.2** As you may have noticed when inspecting the feature matrix above, different features have very different range of values. To make sure features with higher values do not "overwhelm" features with lower values, it is useful to first rescale them all into the [0, 1] range. Run the lines below to carry this out.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
bbbc010_img_feats[:] = scaler.fit_transform(bbbc010_img_feats[:])

display(bbbc010_img_feats)

**1.3** In order to evaluate the quality of our classification attempts, we need a ground truth to compare to. Run the lines below to load and display the ground truth label for each image in the dataset.

In [ ]:
bbbc010_img_gt = pd.read_csv('data/Part 4/BBBC010/bbbc010_image_ground_truth.csv')
bbbc010_img_gt.set_index('image_id', inplace = True)

display(bbbc010_img_gt)

**1.4** To simplify processing later on, it is useful to have a "labelled" version of the feature matrix. Run the lines below to merge the feature matrix and the ground truth labels into a single data frame.

In [ ]:
bbbc010_img = bbbc010_img_feats.join(bbbc010_img_gt)

display(bbbc010_img)

## 2. Feature selection and dimensionality reduction

As we have seen in notebook 3- Quantification, it is hard to find a single feature able to discriminate between dead and live worms. The feature matrix we are working with has thus been built by putting together an extensive collection of 33 measurements capturing shape and intensity, in the hope that all of these features considered together can capture the difference between the 'dead' and 'live' phenotypes. 

While it is clear that more than a single feature is needed, some of the features may however be more informative than others. Among the 33 features considered, some may in fact be entirely uninformative. Revealing which features are relevant and which aren't and making sure that our feature matrix is not too redundant is the job of feature selection and dimensionality reduction methods, as we shall see now.

**2.1** In order to get a first impression of whether each of the feature in our matrix is informative, we can investigate their distributions, for instance by looking at violin plots produced by running the lines below. We know that our dataset is composed of two classes (dead and live), and are therefore mostly interested in features that have a bimodal distribution. Can you spot features that seem to be uninformative?

In [ ]:
# Create a violin plot for each feature
for f in bbbc010_img_feats.keys():
    sb.violinplot(y = bbbc010_img_feats[f])

    plt.show()

**2.2** Variance is a a good proxy for how informative a feature is. Intuitively, any feature that remains constant (or almost constant) throughout the dataset is unlikely to be helpful when it comes to classifying. A simple way to automate the identification of such low-variance features is to set a variance threshold and spot any feature that varies less than that. Run the lines below to load the VarianceThreshold method from scikit-learn and apply it to our feature matrix. Does the result corroboate your observations from 2.1?

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# Define what is considered as too low variance
epsilon = 1e-5
variance_thresh = VarianceThreshold(threshold = epsilon)

# Apply variance threshold
variance_thresh.fit(bbbc010_img_feats)

# Display low-variance features
print(list(bbbc010_img_feats.columns[~variance_thresh.get_support()]))

**2.3** In addition to being informative on their own, we would also like features in our matrix to not be redundant. In other words, we do not want features to correlate too much with each other. This can be investigated by looking at the correlation matrix below. Do you identify groups of features that are heavily correlated? Do you intuitively understand why?

In [ ]:
# Calculate the correlation matrix
correlation_matrix = bbbc010_img_feats.corr()

# Generate a mask for the upper triangle
upper_mask = np.triu(correlation_matrix.corr())

# Visualize the result as a heatmap
palette = sb.diverging_palette(20, 220, n = 256)

sb.heatmap(correlation_matrix, xticklabels=correlation_matrix.columns, yticklabels=correlation_matrix.columns, 
           vmin=-1, vmax=1, center=0, cmap=palette, square=True, mask=upper_mask)

plt.show()

**2.4** When ground-truth class labels are available, Mutual Information provides a formal way of evaluating whether features are predictive of the class label or not. It ranges between 0 when the feature is independent from the class label, to positive values when the feature is dependent on the class, with higher values indicating a stronger dependency. Mutual Information is based on the notion of Shannon entropy, a core theoretical concept in information theory (https://en.wikipedia.org/wiki/Mutual_information). 

Run the lines below to compute the Mutual Information of the features in our matrix and visualize the result. Which ones do you identify as being informative? Does the result corroboate your observations from 2.2?

In [ ]:
from sklearn.feature_selection import mutual_info_classif

In [ ]:
# Create matrix of data and labels
X = bbbc010_img.loc[:, bbbc010_img.columns != 'label']
Y = bbbc010_img.loc[:, 'label']

# Compute feature importance
importance = mutual_info_classif(X,Y)

# Visualize the result
feature_importance = pd.Series(importance, bbbc010_img.columns[0: len(bbbc010_img.columns)-1])
feature_importance.plot(kind='barh', color='teal')

plt.show()

**2.5** Beyond selecting individual features, another way to reduce the dimensionality of our feature matrix is to try and find a few combinations of features that can explain most of the variability present in the matrix. This is the idea behind the famous principal component analysis (PCA, https://en.wikipedia.org/wiki/Principal_component_analysis). 

Run the lines below to 1) extract the first N principal components of our feature matrix, and 2) plot the  variance that they are able to explain. Based on this, how many principal components do you think is sufficient to analyze this dataset? How does that compare to the number of features we initially had, and how does that relate to your observations in 2.1 and 2.3?

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Fix the number of principal components to be extracted
N = 1 ##### Try to modify this #####

# Perform PCA
pca = PCA(n_components = N)
pca.fit(bbbc010_img_feats)

# Retreive the % of variance explained by each Principal Component
percent_variance_explained = np.round(pca.explained_variance_ratio_* 100, decimals=2)
print("Percentage of variance explained with "+str(N)+" principal components: "+str(np.sum(percent_variance_explained)))

# Visualize the explained variance (scree plot)
pc_labels = ['PC'+str(x) for x in range(1, len(percent_variance_explained)+1)]
plt.bar(x = range(1,len(percent_variance_explained)+1), height = percent_variance_explained, 
        tick_label = pc_labels)

plt.ylabel('% Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')

plt.show()

**2.6** PCA, in 2.5, finds a lower-dimensional subspace that maximizes the variation in the whole data set, without accounting for classes. Here, since we also have ground truth labels, we can alternatively apply Linear Discriminant Analysis (LDA, https://en.wikipedia.org/wiki/Linear_discriminant_analysis). LDA aims at retreiving a feature subspace that maximizes the variance between our two classes (*inter-class* variance) while minimizing the variance within each given class (*intra-class* variance). 

Run the lines below to 1) perform LDA on the feature matrix, and 2) plot the data according to the linear discriminant value. Below, we also provides code to plot the data according to the value of their first principal component. Which one of LDA or PCA do you think is better in this case? Can you guess why?

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# Create matrix of data and labels
X = bbbc010_img.loc[:, bbbc010_img.columns != 'label']
Y = bbbc010_img.loc[:, 'label']

# Perform LDA
lda = LinearDiscriminantAnalysis()
bbbc010_img_lda = lda.fit_transform(X, Y)

# Visualize the result
plt.hist(bbbc010_img_lda)

plt.xlabel('LD value')
plt.ylabel('Count')
plt.title('Linear Discriminant Analysis')

plt.show()

In [ ]:
# Perform PCA
pca = PCA(n_components = 1)
pca.fit(bbbc010_img_feats)

# Transform the data into their PC
bbbc010_img_pca = pca.transform(bbbc010_img_feats) 

# Visualize the result
plt.hist(bbbc010_img_pca)

plt.xlabel('1st PC value')
plt.ylabel('Count')
plt.title('Principal Component Analysis')

plt.show()

## 3. Unsupervised classification

Now that we have identified strategies to "clean" our feature matrix, we can dig into the actual classification and try to automatically retreive which group each sample belongs to. We will first look at *unsupervised* classification methods that do not require knowledge of the underlying class labels.

**3.1** K-means clustering (https://en.wikipedia.org/wiki/K-means_clustering) is perhaps the most well-know unsupervised classification algorithm. In a nuthsell, K-means works as follows: 1) K data points are randomly selected as class centroids, 2) all other data points are associated to the centroid they are the closest to, 3) the class centroids are recomputed as the average of the points assigned to them. This procedure is iteratively repeated until convergence, revealing the "true" classes present in the dataset.

Run the lines below to retreive the class label obtained by running 2-means clustering on the LDA-transformed data. Then, adapt the code to run 2-means clustering on the PCA-transformed data, using the 1st principal component only (see 2.6). Do you see a difference?

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# There are two classes in this dataset
K = 2

# Perform K-means on LDA-transformed data
kmeans = KMeans(n_clusters=K, random_state=5)
kmeans.fit(bbbc010_img_lda)

# Retreive the cluster label and display it
kmeans_labels = kmeans.predict(bbbc010_img_lda)

kmeans_labels_lda = np.empty((len(kmeans_labels)), dtype='object')
kmeans_labels_lda[kmeans_labels == 0] = 'dead'
kmeans_labels_lda[kmeans_labels == 1] = 'live'

print(kmeans_labels_lda)

In [ ]:
##### Add you code here for PCA-transformed data ##### 

**3.2** In an unsupervised context we do not have access to class labels. A good way to investigate whether the two clusters we retreived are really two distinct classes or whether they overlap is the Kolmogorov–Smirnov test (KS, https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test). The KS test checks whether the samples from each clusters are drawn from the same distribution or not, giving us an indication of whether two classes indeed exist. The KS test outputs two readouts, the KS statistic (ranging from 0 to 1) and a p-value. A small KS statistic or a high p-value indicate that we cannot reject the null hypothesis stating that the two distributions are equivalent. 

Run the lines below to 1) perform the KS test on the clusters retreived in 3.1, and 2) output the KS statistic  and p-value. What is your conclusion? Adapt the code to run the same analysis on the PCA-transformed data, using the 1st principal component only (see 2.6). Do reach the same conclusion?

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
# Retreive data clusters
cluster_1 = np.squeeze(bbbc010_img_lda[kmeans_labels_lda == 'dead'])
cluster_2 = np.squeeze(bbbc010_img_lda[kmeans_labels_lda == 'live'] )
 
# Perform KS test on LDA-transformed data
stat, pvalue = ks_2samp(cluster_1, cluster_2)

print('Kolmogorov–Smirnov statistic (LDA):', stat)
print('p-value (LDA):', pvalue)
# The two classes indeed form different distributions.

In [ ]:
##### Add you code here for PCA-transformed data ##### 

## 4. Evaluating classification performance

Although we ignored them when carrying out unsupervised classification in part 3, we do in fact have access to class labels. This information can be used to quantitatively evaluate the performance of our classification algorithm.

In [ ]:
import sklearn.metrics as skm

**4.1** The preferred way of reporting the performance of a classifier is the so-called confusion matrix (https://en.wikipedia.org/wiki/Confusion_matrix). Each entry in the matrix tells us how many data points were predicted to belong to the class given by the row index, while actually being from the class indicated by the column index. The confusion matrix therefore allows us to immediately spot the amount of samples that were classified correctly (in the diagonal) or incorrectly (outside of the diagonal).

Run the lines below to compute and display the confusion matrix of the labels predicted by K-means on the LDA-transformed data. Then, adapt the code to obtain the confusion matrix of the labels predicted by K-means on the PCA-transformed data, using the 1st principal component only (see 2.6). Do notice a difference?

In [ ]:
# Retreive predictions (LDA-transformed data) and ground truth 
predictions = kmeans_labels_lda
ground_truth = list(bbbc010_img.loc[:, 'label'])

# Compute the confusion matrix
confusion_matrix = skm.confusion_matrix(ground_truth, predictions, labels = np.unique(ground_truth))

# Visualize the result
disp = skm.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = np.unique(ground_truth))
disp.plot()

plt.show()

In [ ]:
##### Add you code here for PCA-transformed data ##### 

**4.2** Summarizing classification performance as a single number instead of a matrix is sometimes desirable, but often difficult as no metric can captures every single aspect of a classification problem one may want to assess. A large variety of classification metrics are available depending on what one wants to focus on (https://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers).

Run the lines below to compute and display a handful of classification performance metrics evaluating the quality of the labels predicted by K-means on the LDA-transformed data. Then, adapt the code to evaluate the quality of the labels predicted by K-means on the PCA-transformed data, using the 1st principal component only (see 2.6). Do you notice a difference?

In [ ]:
# Retreive predictions (LDA-transformed data) and ground truth
predictions = kmeans_labels_lda
ground_truth = list(bbbc010_img.loc[:, 'label'])

# Compute a handful of classification metrics and display them
accuracy = skm.accuracy_score(ground_truth, predictions)
print('Accuracy: '+str(accuracy))

precision = skm.precision_score(ground_truth, predictions, pos_label= 'live')
print('Precision: '+str(precision))

recall = skm.recall_score(ground_truth, predictions, pos_label= 'live')
print('Recall: '+str(recall))

f1_score = skm.f1_score(ground_truth, predictions, pos_label= 'live')
print('F1 score: '+str(f1_score))

In [ ]:
##### Add you code here for PCA-transformed data ##### 

## 5. Supervised classification

Having access to class labels, we can also explore *supervised* classification methods. These classical (i.e., not "deep") machine learning algorithms use provided labels to learn how features should be combined in order to best discriminate between classes. Once trained, they can be run in *inference mode* on unlabeled data and provide class predictions.

**5.1** A trained classifier is supposed to be able to accurately classify unseen data points, but how to assess whether it does so well enough? The trick is to split our labelled data, prior to training, into a so-called *training* and *test* sets. The data and labels of the training set are visible to the classified and used for the actual training, while the test set remains hidden during that time. Once the classifier is trained, it will have to predict labels for the test set data. These predicted labels will be compared to the known ground-truth ones, thus providing an estimation of the predictive power of the trained algorithm. 

Run the lines below to randomly split the data into a training (2/3 of the data) and test (1/3 of the data) sets. The proportion of samples kept for testing is a free parameter that usually ranges between 1/10 and 1/3.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Create matrix of data and labels
X = bbbc010_img.loc[:, bbbc010_img.columns != 'label']
Y = bbbc010_img.loc[:, 'label']

# Portion of the data for the test set
split = float(1/3)

# Randomly split the data into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(X, Y, test_size = split)

**5.2** Support Vector Machines (SVM, https://en.wikipedia.org/wiki/Support-vector_machine) is a family of classical machine learning algorithms that attempts at finding separation between classes as hyperplanes that are as far as possible to the data points. This is done by maximizing the length of the vector from the closest data point that is orthogonal to the class separation boundary, which is referred to as *support vector* and gives the algorithm its name. 

While the technical details of SVM are out of the scope of this course, we can get a feel for the power of this approach if we try to classify our data as they were originally, without applying any feature selection or dimensionality reduction method. Run the lines below to train a linear SVM on the training set and use it to predict the labels of the test set. How does it look?

In [ ]:
from sklearn import svm

In [ ]:
# Create matrix of data and labels
X = train_data
Y = train_labels

# Train a linear SVM on the training set
classifier = svm.SVC()
classifier.fit(X, Y)

# Predict labels on the test set
pred_labels = classifier.predict(test_data)

# Compute the confusion matrix
confusion_matrix = skm.confusion_matrix(test_labels, pred_labels, labels = np.unique(test_labels))

# Visualize the result
disp = skm.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = np.unique(test_labels))
disp.plot()

plt.show()

**5.3** To compare against SVM, let's try and see how K-means would have fared if it had to deal with the original feature matrix. Adapt the code from 3.1 and 4.1 to run K-means on the whole dataset without dimensionality reduction, and compare with both the K-means result you obtained in 4.1 and the SVM result you obtained in 5.2. What do you notice?

In [ ]:
##### Add you code here ##### 

**5.4** Another popular classical supervised machine learning algorithm is decision trees (https://en.wikipedia.org/wiki/Decision_tree). Decision trees, as their name imply, perform classification by identifying sequences of logical rules to be followed in order to decide which class a sample should be assigned to. In addition to being extremely fast to train, like SVM, decision trees have the interesting property of being fully interpretable. Random forests, as used in ilastik's pixel classification workflow for segmentation (https://www.ilastik.org/), are based on decision trees.

Run the code below to 1) train a decision tree on the training set and retreive predictions on the test set, and 2) visualize the decision tree. Based on this, can you identify which feature(s) is(are) used to decide on the class?

In [ ]:
from sklearn import tree

In [ ]:
# Create matrix of data and labels
X = train_data
Y = train_labels

# Train decision tree on the training set
classifier = tree.DecisionTreeClassifier()
classifier.fit(X, Y)

# Retreive predictions on the test set
pred_labels = classifier.predict(test_data)

# Compute the confusion matrix
confusion_matrix = skm.confusion_matrix(test_labels, pred_labels, labels = np.unique(test_labels))

# Visualize the result
disp = skm.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = np.unique(test_labels))
disp.plot()

plt.show()

In [ ]:
# Visualize the decision tree
tree.plot_tree(classifier)

plt.show()

In [ ]:
# Display the feature used for decision (X[n])
##### Add you code here ##### 

**5.5** The decision tree you trained in 5.4 provides you with a feature and a threshold to classify the data. Try to write a piece of code that assigns class labels by manually thresholding that one feature identified by the decision trees, and check the result. Does that work?

In [ ]:
##### Add you code here ##### 

## BONUS. Classifying individual objects

If you like challenges, apply everything you've seen up to now to try and classify individual *C. elegans* worms in the dataset instead of whole images.

**6.1** Run the lines below to load and display the feature matrix for the entire BBBC010 dataset. This time, features are reported for individual object instances.

In [ ]:
bbbc010_obj_feats = pd.read_csv('data/Part 4/BBBC010/bbbc010_object_features.csv')
bbbc010_obj_feats.set_index('instance_id', inplace = True)

display(bbbc010_obj_feats)

**6.2** Run the lines below to load and display the ground truth label for each instance in the dataset.

In [ ]:
bbbc010_obj_gt = pd.read_csv('data/Part 4/BBBC010/bbbc010_object_ground_truth.csv')
bbbc010_obj_gt.set_index('instance_id', inplace = True)

display(bbbc010_obj_gt)

**6.3** Classify individual worms in the dataset as "dead" or "live", and report the performance of your solution using appropriate metrics. 

In [ ]:
##### Add you code here ##### 

**6.4** Based on your classification results, identify images that contain a mix of dead and live worms. Relating your observations with the whole-image label they had in bbbc010_img_gt (1.3), would you say that these images were mislabeled? Why?

In [ ]:
##### Add you code here ##### 